In [13]:
#!pip install mesa seaborn --quiet

In [1]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import random

In [ ]:
class PortraitAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.is_revealed = False
    self.is_fake = False
    self.damaged = False

  def reveal(self):
    if not self.is_revealed:
      self.is_revealed = True
      if self.is_fake:
        print(f"El retrato {self.unique_id} ha sido revelado como un espejo (falsa alarma) en {self.pos}")
      else:
        print(f"El retrato {self.unique_id} ha sido revelado como un personaje en {self.pos}")

  def damage(self):
      if not self.damaged:
        self.damaged = True
        print(f"El retrato {self.unique_id} ha sufrido daño en {self.pos}")
      else:
        print(f"El retrato {self.unique_id} ya estaba dañado.")

In [ ]:
class GhostAgent(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.points = 1 # Todos los fantasmas empiezan con 1 punto (Green Ghost)
    self.is_dead = False # Para determinar si un fantasma sigue en el tablero

  # Si Luigi decide noquear a un fantasma (-1 punto)
  def is_knocked(self):
    if not self.dead:
      self.points -= 1
      print(f"El fantasma {self.unique_id} ha sido noqueado.")
      self.check_health()

  # Si Luigi decide absorber a un fantasma (-2 puntos)
  def is_absorbed(self):
    if not self.dead:
      self.points -= 2
      self.is_dead = True
      print(f"El fantasma {self.unique_id} ha sido absorbido.")

  def check_health(self):
    if self.points <= 0: 
      self.dead = True
      print(f"El fantasma {self.unique_id} ha sido eliminado.")

  # Si es Green Ghost, se vuelve Red Ghost el siguiente turno
  def get_red(self):
      # Volverse Red Ghost
      self.points = 2
      print(f"El fantsma {self.unique_id} se ha convertido en un fantasma rojo.")

In [ ]:
class LuigiAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.points = 4

    def get_adjacent_agents(self):
        # Obtener los agentes en las celdas adyacentes
        adjacent_agents = []
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)

        for pos in possible_moves:
            adjacent_agents.extend(self.model.grid.get_cell_list_contents([pos]))
        
        return adjacent_agents

    def decide_to_save_points(self):
        # Decidir aleatoriamente si ahorrar puntos
        return random.choice([True, False])  # 50% de probabilidad de ahorrar

    def interact(self):
        adjacent_agents = self.get_adjacent_agents()
        
        for agent in adjacent_agents:
            if isinstance(agent, GhostAgent):
                # Decide si interactuar con el fantasma
                if self.points >= 2:  # Puede exorcizar
                    self.perform_exorcism(agent)
                elif self.points >= 1:  # Puede noquear
                    self.perform_knocked(agent)
            elif isinstance(agent, PortraitAgent):
                # Decide recoger el retrato si hay puntos
                if self.points >= 1:
                    self.collect_portrait(agent)

    def move(self):
        if self.points > 0:
            possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
            new_position = self.random.choice(possible_moves)
            self.model.grid.move_agent(self, new_position)
            self.points -= 1  # Costo de moverse

    def perform_knocked(self, ghost):
        if self.points >= 1:  # Costo de noquear
            ghost.is_knocked(ghost)
            self.points -= 1
            print(f"Luigi {self.unique_id} ha noqueado un fantasma en {self.pos}")

    def perform_exorcism(self, ghost):
        if self.points >= 2 and ghost.points == 2:  # Costo de exorcizar
            ghost.is_absorbed(ghost)
            self.points -= 2
            print(f"Luigi {self.unique_id} ha absorbido un fantasma en {self.pos}")

    def collect_portrait(self, portrait):
        portrait.recovered = True
        self.points -= 1  # Costo de recoger el artefacto
        print(f"Luigi {self.unique_id} ha recogido un retrato en {self.pos}")

    def step(self):
        if not self.decide_to_save_points():
            self.move()  # Si no decide ahorrar, se mueve
        self.interact()  # Realiza interacciones

In [ ]:
def get_grid(model):
    # Inicializamos una cuadrícula vacía
    grid = np.empty((model.grid.width, model.grid.height), dtype=object)
    
    # Iteramos sobre todas las celdas de la cuadrícula
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            # Obtenemos los agentes en la celda actual
            agents_in_cell = model.grid.get_cell_list_contents([(x, y)])
            
            # Asignamos una lista de agentes en esa celda (por ejemplo: fantasmas, retratos, Luigi)
            grid[x, y] = agents_in_cell
            
    return grid


In [ ]:
class MansionModel(Model):
    def __init__(self, width, height, num_food=20, num_agents=5, max_energy=10):
        super().__init__()
        self.grid = SingleGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            model_reporters={
                "Grid": get_grid,
                "Steps": lambda model: model.steps,
            },
        )
        self.num_agents = num_agents  # Número de agentes fantasmas y retratos
        self.max_energy = max_energy
        self.create_agents()

    def create_agents(self):
        # Crear fantasmas y retratos directamente dentro del modelo
        for i in range(self.num_agents):
            # Fantasmas
            ghost_points = 1  # Fantasma inicial con 1 punto (Green Ghost)
            ghost_is_dead = False  # El fantasma comienza vivo
            ghost_pos = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
            self.grid.place_agent(ghost_points, ghost_is_dead, ghost_pos, "GhostAgent")

            # Retratos
            portrait_is_revealed = False
            portrait_is_fake = False  # Puede ser ajustado más tarde si es necesario
            portrait_damaged = False
            portrait_pos = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
            self.grid.place_agent(portrait_is_revealed, portrait_is_fake, portrait_damaged, portrait_pos, "PortraitAgent")

        # Crear Luigi
        luigi_pos = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
        self.grid.place_agent(luigi_pos, "LuigiAgent")

    def get_grid(self, model):
        # Devuelve una representación de la cuadrícula
        grid = np.zeros((model.grid.width, model.grid.height), dtype=object)
        
        for cell in range(model.grid.width):
            for row in range(model.grid.height):
                cell_content = model.grid.get_cell_list_contents([(cell, row)])
                grid[cell][row] = cell_content  # Guarda los agentes de la celda
        return grid

    def step(self):
        # Realiza un paso en la simulación, actualizando el estado de todos los agentes
        for agent in self.grid.get_all_agents():
            if isinstance(agent, LuigiAgent):
                agent.step()  # Los agentes Luigi realizan su acción
            elif isinstance(agent, "GhostAgent"):
                self.handle_ghost(agent)  # Maneja la lógica de fantasmas
            elif isinstance(agent, "PortraitAgent"):
                self.handle_portrait(agent)  # Maneja la lógica de retratos
        self.datacollector.collect(self)

    def handle_ghost(self, ghost):
        # Lógica para manejar los fantasmas dentro del modelo
        if ghost["is_dead"]:
            return  # Si el fantasma está muerto, no hacer nada
        if ghost["points"] <= 0:
            ghost["is_dead"] = True
            print(f"El fantasma en {ghost['pos']} ha sido eliminado.")

    def handle_portrait(self, portrait):
        # Lógica para manejar los retratos dentro del modelo
        if portrait["is_revealed"]:
            if portrait["is_fake"]:
                print(f"El retrato en {portrait['pos']} ha sido revelado como falso.")
            else:
                print(f"El retrato en {portrait['pos']} ha sido revelado correctamente.")
        if portrait["damaged"]:
            print(f"El retrato en {portrait['pos']} ha sido dañado.")
